In [1]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [51]:
recipe = pd.read_csv('./data/recipe.csv')
recipe.tail()

,recipe_id,recipe_name,recipe_method,recipe_kind,recipe_difficulty,recipe_rating,recipe_ingredient,recipe_kcal,recipe_carbohydrate,recipe_protein,recipe_fat,recipe_natrium,recipe_thumbnail,recipe_bittersweet,recipe_spicy,recipe_goso,recipe_oily,recipe_sweetsour,recipe_plain
1303,recipe-1304,맥적구이,굽기,반찬,0,0,"돼지고기(120g), 생강(10g), 소금(0.2g), 후춧가루(0.02g)\r\n...",467.7,17.60,30.7,30.5,551.7,http://www.foodsafetykorea.go.kr/uploadimg/coo...,0,0,0,0,0,0
1304,recipe-1305,채소어묵,튀기기,반찬,0,0,"생선살(100g), 맛술(10g), 생강(10g), 소금(0.2g)\r\n밀가루(2...",382.5,39.89,26.7,12.9,320.1,http://www.foodsafetykorea.go.kr/uploadimg/coo...,0,0,0,0,0,0
1305,recipe-1306,주꾸미보리죽,끓이기,밥,0,0,"주꾸미(100g), 밀가루(10g), 보리(100g)\r\n미나리(10g), 당근(...",327.4,48.70,21.8,5.0,172.7,http://www.foodsafetykorea.go.kr/uploadimg/coo...,0,0,0,0,0,0
1306,recipe-1307,오렌지삼겹찜,굽기,반찬,0,0,"삼겹살(100g), 로즈마리(5g)\r\n고구마(30g), 생크림(10g), 감자(...",638.1,22.00,26.7,49.2,206.0,http://www.foodsafetykorea.go.kr/uploadimg/coo...,0,0,0,0,0,0
1307,recipe-1308,소고기볼게살타르,굽기,반찬,0,0,"다진 소고기(100g), 마늘(10g), 양파(20g), 파프리카(20g)\r\n후...",418.0,31.80,24.8,21.3,245.8,http://www.foodsafetykorea.go.kr/uploadimg/coo...,0,0,0,0,0,0


In [53]:
recipe = recipe[['recipe_id', 'recipe_name']]
recipe.tail()

,recipe_id,recipe_name
1303,recipe-1304,맥적구이
1304,recipe-1305,채소어묵
1305,recipe-1306,주꾸미보리죽
1306,recipe-1307,오렌지삼겹찜
1307,recipe-1308,소고기볼게살타르


In [54]:
length = len(recipe)

In [58]:
# user_id, rating, recipe_id
ratings = pd.read_csv('./data/ratings.csv')
ratings = ratings[['ratingNo','recipe_id', 'user_id', 'rating']]
ratings.tail()

,ratingNo,recipe_id,user_id,rating
39184,39184,recipe-1308,user-88,2
39185,39185,recipe-1308,user-89,2
39186,39186,recipe-1308,user-91,2
39187,39187,recipe-1308,user-94,1
39188,39188,recipe-1308,user-97,3


In [18]:
# user_id
users = pd.read_csv('./data/user.csv')
users = users[['ratingNo','user_id']]
users.tail()

,ratingNo,user_id
95,95,user-96
96,96,user-97
97,97,user-98
98,98,user-99
99,99,user-100


In [19]:
del ratings['ratingNo']
ratings.tail()

,recipe_id,user_id,rating
39184,recipe-1308,user-88,2
39185,recipe-1308,user-89,2
39186,recipe-1308,user-91,2
39187,recipe-1308,user-94,1
39188,recipe-1308,user-97,3


In [20]:
user_num = ratings.user_id.unique().shape[0]
recipe_num = ratings.recipe_id.unique().shape[0]

print("유저의 수는 {}명 이고, \n레시피의 수는 {}개 입니다.".format(user_num, recipe_num))

유저의 수는 100명 이고, 
레시피의 수는 1308개 입니다.


In [70]:
# 사용자 샘플 선출 -> pivot table 생성
UM_matrix_ds = ratings.pivot(index='user_id',columns='recipe_id', values='rating')
UM_matrix_ds

recipe_id,recipe-1,recipe-10,recipe-100,recipe-1000,recipe-1001,recipe-1002,recipe-1003,recipe-1004,recipe-1005,recipe-1006,...,recipe-990,recipe-991,recipe-992,recipe-993,recipe-994,recipe-995,recipe-996,recipe-997,recipe-998,recipe-999
user_id,,,,,,,,,,,,,,,,,,,,,
user-1,3.0,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,...,5.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
user-10,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,4.0,4.0,NaN,1.0
user-100,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
user-11,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN
user-12,1.0,4.0,NaN,2.0,NaN,NaN,2.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,2.0,NaN,5.0,4.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
user-95,1.0,NaN,NaN,2.0,NaN,2.0,1.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
user-96,2.0,NaN,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user-97,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [80]:
def distance_cosine(a, b) :
       return 1-distance.cosine(a, b)

In [81]:
def distance_correlation(a, b) :
       return 1-distance.correlation(a, b)

In [82]:
def distance_euclidean(a, b) :
       return 1/(1+distance.euclidean(a, b))

In [83]:
def nearest_neighbor_user(user, topN, simFunc) :
    u1  =  UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = { }
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = [ ]
        interSectionU2 = [ ]
        if uid == user: continue
        
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim):nn[uid] = sim
                                                
#     return sorted(nn.items(), key=itemgetter(1))[::-1]
    return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]

In [86]:
for user in range(0, len(UM_matrix_ds)) :
      print('User {0} neighbors : {1}'.format('user-'+str(user+1), nearest_neighbor_user('user-'+str(user+1), 3, distance_cosine)))

User user-1 neighbors : [('user-79', 0.8873525883018792), ('user-75', 0.8826151617831657), ('user-65', 0.8637526728156629)]
User user-2 neighbors : [('user-42', 0.879597155335769), ('user-82', 0.8635368667338414), ('user-53', 0.8558833722757675)]
User user-3 neighbors : [('user-16', 0.8849578627702183), ('user-43', 0.8565254968724393), ('user-19', 0.8541606139626727)]
User user-4 neighbors : [('user-93', 0.8804669431007153), ('user-25', 0.8622221277213945), ('user-49', 0.8611766090660558)]
User user-5 neighbors : [('user-9', 0.8729551763737453), ('user-96', 0.8619480438685505), ('user-55', 0.8598252252087156)]
User user-6 neighbors : [('user-42', 0.8616934300748349), ('user-52', 0.8608053305542304), ('user-60', 0.8538180221532102)]
User user-7 neighbors : [('user-25', 0.85888055455711), ('user-76', 0.8568888888888889), ('user-2', 0.8548594734655425)]
User user-8 neighbors : [('user-66', 0.8714667817894), ('user-46', 0.8662915739369857), ('user-1', 0.862390171890171)]
User user-9 neighb

User user-67 neighbors : [('user-64', 0.8589224129237321), ('user-16', 0.8574532279388077), ('user-57', 0.8475473819321789)]
User user-68 neighbors : [('user-34', 0.8662524225388307), ('user-94', 0.8554535354989778), ('user-81', 0.8543634315367674)]
User user-69 neighbors : [('user-57', 0.8607597439252483), ('user-56', 0.8583869296201797), ('user-1', 0.8519206780307919)]
User user-70 neighbors : [('user-50', 0.8642479194345328), ('user-53', 0.8603009936007606), ('user-76', 0.8553031904486218)]
User user-71 neighbors : [('user-25', 0.8625154147178834), ('user-56', 0.8559314490528386), ('user-82', 0.8511418499493604)]
User user-72 neighbors : [('user-86', 0.859870120093148), ('user-50', 0.8563265178412649), ('user-8', 0.853324932529605)]
User user-73 neighbors : [('user-61', 0.8649110061097874), ('user-21', 0.8594589415600048), ('user-5', 0.859090298663252)]
User user-74 neighbors : [('user-84', 0.8725199116971338), ('user-61', 0.8692822075696746), ('user-46', 0.8618257940290289)]
User u

In [90]:
def predict_rating(userid, nn=100, simFunc=distance_cosine):
    neighbor = nearest_neighbor_user(userid, nn, distance_cosine)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_recipe = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh=1)
    neighbor_dict = (dict(neighbor))
    ret = []
    
    for recipe_id, row in neighbor_recipe.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0], 0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([recipe_id, wsum/jsum])
        
    return ret

In [92]:
is_users = ratings['user_id'] == 'user-1'
users_rating = ratings[is_users]
users_rating['recipe_id']

rating_list = users_rating['recipe_id'].tolist()
rating_list

if 'recipe-10' not in rating_list:
    print("없다")

'user-1'

In [126]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 10, distance_cosine)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        if recipe[0] not in exist_recipe:
            result_list.append([user_id, recipe[0], recipe[1]])
#         result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf

,user_id,recipe_id,rating
0,user-1,recipe-10,4.000000
1,user-1,recipe-100,2.498911
2,user-1,recipe-1000,4.496946
3,user-1,recipe-1002,2.000000
4,user-1,recipe-1003,2.023335
...,...,...,...
88715,user-100,recipe-993,2.613003
88716,user-100,recipe-994,5.000000
88717,user-100,recipe-995,2.667726
88718,user-100,recipe-997,2.807082


In [129]:
resultdf.tail()

,user_id,recipe_id,rating
88715,user-100,recipe-993,2.613003
88716,user-100,recipe-994,5.000000
88717,user-100,recipe-995,2.667726
88718,user-100,recipe-997,2.807082
88719,user-100,recipe-999,1.502072


In [133]:
resultdf.to_csv('C:\\SSAFY\\s04p23b205\\backend\\backend_data\\data\\predict.csv', sep=',')

In [131]:
resultdf.dtypes

user_id       object
recipe_id     object
rating       float64
dtype: object

In [140]:
for userid in resultdf['user_id'].unique():
    print(ratings[ratings['user_id']==userid]['rating'])

0        3
466      5
557      2
591      3
780      3
        ..
38796    2
38815    5
38845    1
39044    5
39155    5
Name: rating, Length: 308, dtype: int64
153      5
411      2
592      3
753      3
862      4
        ..
38476    3
38619    3
38797    3
38986    1
39100    1
Name: rating, Length: 323, dtype: int64
27       4
85       3
266      1
349      5
492      3
        ..
38381    5
38730    5
38950    4
39013    5
39131    1
Name: rating, Length: 375, dtype: int64
154      3
206      5
294      3
593      5
896      3
        ..
38562    2
38620    1
38867    5
38924    4
39045    1
Name: rating, Length: 346, dtype: int64
230      3
350      4
412      3
442      5
619      3
        ..
38594    5
38621    3
38675    4
38987    1
39014    4
Name: rating, Length: 332, dtype: int64
86       1
155      2
231      3
267      5
316      1
        ..
38676    2
38798    5
38816    1
38951    1
39156    3
Name: rating, Length: 446, dtype: int64
87       2
121      5
156      1
2

In [192]:
error_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 40, distance_cosine)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        if recipe[0] in exist_recipe:
            error_list.append([user_id, recipe[0], recipe[1]])
#         result_list.append([user_id, recipe[0], recipe[1]])
    
errordf = pd.DataFrame(error_list, columns=['user_id','recipe_id','rating'])
errordf

,user_id,recipe_id,rating
0,user-1,recipe-1,3.494289
1,user-1,recipe-1001,2.716333
2,user-1,recipe-1004,2.745370
3,user-1,recipe-1008,3.176988
4,user-1,recipe-1013,3.397204
...,...,...,...
39184,user-100,recipe-98,2.696866
39185,user-100,recipe-980,3.082625
39186,user-100,recipe-989,3.606289
39187,user-100,recipe-992,3.009187


In [193]:
predict = pd.merge(left = ratings[['recipe_id','user_id','rating']], right = errordf, on=['user_id','recipe_id'])

In [185]:
predict

,recipe_id,user_id,rating_x,rating_y
0,recipe-1,user-1,3,3.326241
1,recipe-1,user-10,4,2.790280
2,recipe-1,user-12,1,2.593933
3,recipe-1,user-15,1,3.994855
4,recipe-1,user-19,4,1.803402
...,...,...,...,...
39178,recipe-1308,user-88,2,2.143269
39179,recipe-1308,user-89,2,2.149402
39180,recipe-1308,user-91,2,2.110321
39181,recipe-1308,user-94,1,2.326779


In [194]:
realdata_rating = predict['rating_x'].tolist()
resultdata_rating = predict['rating_y'].tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)
print("\nError Rate(Absolute) : ", error_rate_absol)  # MAE
print("Error Rate(Squared) : ", error_rate_squared)  # MSE
print("RMSE : ", math.sqrt(error_rate_squared))  # RMSE


Error Rate(Absolute) :  1.1773392842216803
Error Rate(Squared) :  1.8889840913932743
RMSE :  1.3744031764345113
